In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.model.pointwise.model import HomophilyContentCNN
from src.utils.utils import create_node2vec_embedding_layer
from src.constants import MAX_LENGTH_USER_REPRESENATION, MAX_LENGTH_COMMENT_SECTION, WORD_EMBEDDING_PATH
import torch
from collections import OrderedDict

In [ ]:
user_embedding_path = ''
path = ''

In [ ]:
NODE2VEC_EMB_DIM, num_authors, node2vec_emb_layer, author_to_pos_dict = create_node2vec_embedding_layer(
        user_embedding_path, True)

In [ ]:
model = HomophilyContentCNN(node2vec_emb_layer,
                                NODE2VEC_EMB_DIM,
                                MAX_LENGTH_USER_REPRESENATION,
                                MAX_LENGTH_COMMENT_SECTION,
                           user_num_kernels=100,
                           section_num_kernels=100,
                           user_kernel_size=4,
                            section_kernel_size=4,
                            latent_factors_deepconn=128
                           )

In [ ]:

checkpoint = torch.load(path)
config = checkpoint['config']
state_dict = checkpoint['state_dict']

del config['pairwise']
del config['learning_rate']
del config['batch_size']

In [ ]:
print(config)

In [ ]:
state_dict.keys()

In [ ]:
model_dict = model.state_dict()

In [ ]:
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:]  # remove `module.`
    new_state_dict[name] = v
# load params

del new_state_dict['shared_linear.0.weight']
pretrained_dict = {k: v for k, v in new_state_dict.items() if k in model_dict}
print(pretrained_dict.keys())
model_dict.update(pretrained_dict) 
model.load_state_dict(model_dict)

In [ ]:
model_dict = model.state_dict()

In [ ]:
model_dict['shared_linear.0.weight']